In [1]:
import requests
from bs4 import BeautifulSoup
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

In [2]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sara_\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [17]:
def fetch_article_content(url):
    """Fetch the article content from the given URL."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the content wrapper
    content_wrapper = soup.find('div', class_='content_wrapper arti-flow')
    if content_wrapper:
        # Extract the main content
        paragraphs = content_wrapper.find_all('p')
        article_content = ' '.join(paragraph.text.strip() for paragraph in paragraphs)
        return article_content
    return 'Content not found.'

In [18]:
def fetch_news_articles(keyword, max_pages=30):  # Set max_pages to the desired number of pages to scrape
    articles = []

    # Loop through the pages
    for page in range(1, max_pages + 1):
        url = f"https://www.moneycontrol.com/news/business/stocks/page-{page}/"
        
        # Send a request to fetch the page
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all relevant news items
        for item in soup.find_all('li', class_='clearfix'):
            title_tag = item.find('h2').find('a')  # 'a' tag contains the link and title
            if title_tag:
                title = title_tag.text.strip()
                link = title_tag['href']

                # Check if the keyword is in the headline
                if keyword.lower() in title.lower():
                    article_url = link if link else ''
                    
                    # Fetch the article summary
                    summary_tag = item.find('p')  # Paragraph tag contains the summary
                    summary = summary_tag.text.strip() if summary_tag else 'No summary available.'

                    if summary:  # Only add articles with a summary
                        # Fetch the article content
                        content = fetch_article_content(article_url)

                        articles.append((title, summary, article_url, content))

    return articles

In [19]:
def analyze_sentiment(text):
    """Analyze sentiment of the given text and return individual scores."""
    sia = SentimentIntensityAnalyzer()
    scores = sia.polarity_scores(text)

    # Return scores for bearish, neutral, and bullish
    return {
        "bearish": scores['neg'],
        "neutral": scores['neu'],
        "bullish": scores['pos'],
        "compound": scores['compound']
    }


In [20]:
def sentiment_analysis_on_articles(articles):
    """Analyze sentiment for each article and return results with average scores."""
    results = []
    total_scores = {"bearish": 0, "neutral": 0, "bullish": 0}
    article_count = len(articles)

    for title, summary, url, content in articles:
        combined_text = f"{title} {summary} {content}"  # Combine title, summary, and content
        sentiment_scores = analyze_sentiment(combined_text)  # Analyze sentiment
        
        # Check if the returned value is a dictionary
        if isinstance(sentiment_scores, dict):
            results.append((title, summary, url, content, sentiment_scores))  # Append results

            # Accumulate scores for averaging
            total_scores["bearish"] += sentiment_scores["bearish"]
            total_scores["neutral"] += sentiment_scores["neutral"]
            total_scores["bullish"] += sentiment_scores["bullish"]
        else:
            print(f"Error analyzing sentiment for article: {title}. Result: {sentiment_scores}")

    # Calculate average scores
    if article_count > 0:
        avg_scores = {
            "bearish": total_scores["bearish"] / article_count,
            "neutral": total_scores["neutral"] / article_count,
            "bullish": total_scores["bullish"] / article_count
        }
    else:
        avg_scores = {"bearish": 0, "neutral": 0, "bullish": 0}

    return results, avg_scores

In [21]:
keyword = input("Enter a keyword to search for news: ")
articles = fetch_news_articles(keyword)

# Perform sentiment analysis on the fetched articles
if articles:
    analyzed_articles, average_scores = sentiment_analysis_on_articles(articles)
    
    # Print the sentiment results for each article
    for title, summary, url, content, sentiment_scores in analyzed_articles:
        print(f"Title: {title}")
        print(f"Summary: {summary}")
        print(f"URL: {url}")
        print(f"Content: {content[:100]}...")  # Display first 100 characters of content
        print(f"Bearish Score: {sentiment_scores['bearish']:.4f}")
        print(f"Neutral Score: {sentiment_scores['neutral']:.4f}")
        print(f"Bullish Score: {sentiment_scores['bullish']:.4f}\n")
    
    # Display average sentiment scores for the keyword
    print("Average Sentiment Scores:")
    print(f"Bearish: {average_scores['bearish']:.4f}")
    print(f"Neutral: {average_scores['neutral']:.4f}")
    print(f"Bullish: {average_scores['bullish']:.4f}")
else:
    print("No articles found with the given keyword.")

Title: Hold Tata Communication; target of Rs 1855: ICICI Securities
Summary: ICICI Securities recommended hold rating on Tata Communication with a target price of Rs 1855 in its research report dated October 18, 2024.
URL: https://www.moneycontrol.com/news/business/stocks/hold-tata-communication-target-of-rs-1855-icici-securities-12844924.html
Content: ICICI Securities research report on Tata Communication Tata Communications’ (TCom) H1FY25 revenue pr...
Bearish Score: 0.0000
Neutral Score: 0.8540
Bullish Score: 0.1460

Title: Morgan Stanley 'Underweight' on Tata Chemicals stock after Q2 profit plunges 55%
Summary: Tata Chemicals' Q2 profit plunged 55 percent, hurt by falling soda ash prices and rising freight costs, leading Morgan Stanley to maintain an 'Underweight' rating with a target price of Rs 880.
URL: https://www.moneycontrol.com/news/business/markets/morgan-stanley-underweight-on-tata-chemicals-stock-after-q2-profit-plunges-55-12844524.html
Content: Shares of Tata Chemicals f